In [1]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
import time

# Load nodes and edges from CSV files
nodes_df = pd.read_csv('ICS_OT NodesInfected.csv')
edges_df = pd.read_csv('ICS_OT EdgesInfected.csv')

# Create a NetworkX graph
G = nx.Graph()

# Add nodes with dummy features (since only Id and Label are provided)
for _, row in nodes_df.iterrows():
    G.add_node(row['Id'], label=row['Label'], feature1=1.0, feature2=1.0)  # Example features

# Add edges
for _, row in edges_df.iterrows():
    G.add_edge(row['Source'], row['Target'])

# Convert NetworkX graph to PyTorch Geometric data format
x = torch.tensor([[1.0, 1.0] for _ in range(len(nodes_df))], dtype=torch.float32)  # Dummy features
edge_index = torch.tensor(edges_df[['Source', 'Target']].values.T, dtype=torch.long)

# Ensure all indices are within the valid range of node indices
valid_edges_mask = (edge_index[0] < len(nodes_df)) & (edge_index[1] < len(nodes_df))
edge_index = edge_index[:, valid_edges_mask]

# Create dummy labels for the nodes
labels = torch.tensor([0 for _ in range(len(nodes_df))], dtype=torch.long)  # Example labels

# Split data into training and test sets
train_mask, test_mask = train_test_split(range(len(labels)), test_size=0.2, random_state=42)

data = Data(x=x, edge_index=edge_index, y=labels)
data.train_mask = torch.tensor(train_mask, dtype=torch.long)
data.test_mask = torch.tensor(test_mask, dtype=torch.long)

# Define the GCN model using spectral convolutions
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(in_channels=2, hidden_channels=4, out_channels=2)  # Adjust parameters as needed

# Set up the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Record the start time
start_time = time.time()

# Training the model
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f'Training completed in {elapsed_time:.2f} seconds')

# Evaluate the model
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out[data.test_mask].argmax(dim=1)
    accuracy = (pred == data.y[data.test_mask]).sum().item() / len(data.test_mask)
    print(f'Test Accuracy: {accuracy:.4f}')

Epoch 0, Loss: 0.6931472420692444
Epoch 10, Loss: 0.598453164100647
Epoch 20, Loss: 0.5151417851448059
Epoch 30, Loss: 0.4435594975948334
Epoch 40, Loss: 0.38309967517852783
Epoch 50, Loss: 0.33254337310791016
Epoch 60, Loss: 0.29043474793434143
Epoch 70, Loss: 0.2553446292877197
Epoch 80, Loss: 0.22600261867046356
Epoch 90, Loss: 0.2013387680053711
Training completed in 0.17 seconds
Test Accuracy: 1.0000
